In [1]:
!pip install groq requests

from groq import Groq
import json
import re
API_KEY = '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'

client = Groq(
    api_key=API_KEY,
    default_headers={"Groq-Model-Version": "latest"}
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 2.8 MB/s eta 0:00:00


In [2]:
conversation_history = []

def add_to_history(role, message):
    conversation_history.append({"role": role, "message": message})

def get_recent_history(n_turns=None, max_chars=None):
    history = conversation_history[-n_turns:] if n_turns else conversation_history[:]
    history_text = '\n'.join([f"{entry['role']}: {entry['message']}" for entry in history])
    return history_text[-max_chars:] if max_chars and len(history_text) > max_chars else history_text


In [3]:
SUMMARY_INTERVAL = 3

def summarize_history(history_text):
    response = client.chat.completions.create(
        model="groq/compound",
        messages=[
            {"role": "user", "content": f"Summarize the following conversation concisely:\n{history_text}"}
        ],
        temperature=0.5,
        max_completion_tokens=150
    )
    return response.choices[0].message.content.strip()


In [4]:
def handle_conversation(role, message):
    global conversation_history
    add_to_history(role, message)
    print(f"{role}: {message}")

    if len(conversation_history) % SUMMARY_INTERVAL == 0:
        recent_history = get_recent_history()
        summary = summarize_history(recent_history)
        conversation_history = [{"role": "system", "message": f"Summary so far: {summary}"}]
        print("\n--- Summarization Performed ---")
        print(f"Summary: {summary}\n")


In [6]:
handle_conversation("User", "Hi, I want to learn Data Science.")
handle_conversation("Assistant", "Sure! Do you have programming experience?")
handle_conversation("User", "Yes, some Python basics.")
handle_conversation("Assistant", "Great, start with pandas and numpy.")
handle_conversation("User", "Recommend online courses please.")
handle_conversation("Assistant", "Coursera and Udemy have good options.")
print("\nFinal Conversation History:")
for entry in conversation_history:
    print(f"{entry['role']}: {entry['message']}")


User: Hi, I want to learn Data Science.

--- Summarization Performed ---
Summary: **Summary:** The user wants to learn Data Science and has basic Python skills. The assistant advised starting with pandas and NumPy and suggested looking at Coursera and Udemy for relevant courses. The user reiterated their interest in learning Data Science.

Assistant: Sure! Do you have programming experience?
User: Yes, some Python basics.

--- Summarization Performed ---
Summary: The user wants to learn Data Science and has basic Python knowledge. The assistant recommended beginning with pandas and NumPy and exploring Coursera/Udemy courses, then asked if the user has programming experience.

Assistant: Great, start with pandas and numpy.
User: Recommend online courses please.

--- Summarization Performed ---
Summary: The user wants to learn Data Science with basic Python skills; the assistant suggested starting with pandas and NumPy, recommended looking at Coursera/Udemy courses, asked about the user’

In [7]:
expected_schema = {
    "name": str,
    "email": str,
    "phone": str,
    "location": str,
    "age": int
}


In [8]:
def extract_json_from_response(response_text):
    json_match = re.search(r"```json\s*(\{.*?\})\s*```", response_text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return {"error": "Failed to parse JSON", "raw_response": response_text}
    else:
        try:
            return json.loads(response_text)
        except json.JSONDecodeError:
            return {"error": "Invalid JSON format", "raw_response": response_text}


In [9]:
def extract_info_from_chat(chat_text):
    response = client.chat.completions.create(
        model="groq/compound",
        messages=[
            {"role": "user", "content": f"Extract in JSON format: name, email, phone, location, age from this chat:\n{chat_text}"}
        ],
        temperature=0,
        max_completion_tokens=150
    )

    result_text = response.choices[0].message.content.strip()
    return extract_json_from_response(result_text)



In [10]:
def validate_extracted_data(data):
    if "error" in data:
        return data

    for key, expected_type in expected_schema.items():
        if key not in data:
            return {"error": f"Missing key: {key}"}
        if not isinstance(data[key], expected_type):
            return {"error": f"Incorrect type for {key}, expected {expected_type.__name__}"}
    return {"success": True, "data": data}


In [11]:
sample_chats = [
    "My name is John Doe, email is john@example.com, phone 1234567890, I live in New York, age 30.",
    "I'm Jane from California. Contact: jane.doe@example.com, 0987654321. I'm 28 years old.",
    "Hi, Mark here. Email: mark123@gmail.com, Phone: 1122334455, Location: Texas, Age: 35."
]

for i, chat in enumerate(sample_chats):
    extracted = extract_info_from_chat(chat)
    validation = validate_extracted_data(extracted)

    print(f"\nSample Chat {i+1}:")
    print(f"Extracted Data: {extracted}")
    print(f"Validation Result: {validation}")



Sample Chat 1:
Extracted Data: {'name': 'John Doe', 'email': 'john@example.com', 'phone': '1234567890', 'location': 'New York', 'age': 30}
Validation Result: {'success': True, 'data': {'name': 'John Doe', 'email': 'john@example.com', 'phone': '1234567890', 'location': 'New York', 'age': 30}}

Sample Chat 2:
Extracted Data: {'name': 'Jane', 'email': 'jane.doe@example.com', 'phone': '0987654321', 'location': 'California', 'age': 28}
Validation Result: {'success': True, 'data': {'name': 'Jane', 'email': 'jane.doe@example.com', 'phone': '0987654321', 'location': 'California', 'age': 28}}

Sample Chat 3:
Extracted Data: {'name': 'Mark', 'email': 'mark123@gmail.com', 'phone': '1122334455', 'location': 'Texas', 'age': 35}
Validation Result: {'success': True, 'data': {'name': 'Mark', 'email': 'mark123@gmail.com', 'phone': '1122334455', 'location': 'Texas', 'age': 35}}
